# LATE推定量向け検定シミュレーション・サンプルサイズ設計

## モジュールインポート

In [7]:
import numpy as np
import pandas as pd
from scipy import stats
from statsmodels.api import OLS, add_constant
from statsmodels.stats.weightstats import ttest_ind
from tqdm import tqdm

In [17]:
# シミュレーション用のデータを作成
n = 100              # データのサイズ
p_value_thres = 0.05 # 有意水準
n_trials = 10000     # 試行回数

significant_list = []
for i in tqdm(range(n_trials)):
    significant_flag = 0
    np.random.seed(i) # ランダムシード

    # ツール変数Z (0 or 1, ランダムに生成)
    Z = np.random.choice([0, 1], size=N)
    
    # 処置変数D: Zに応じて多少の影響を受ける（例：コンプライアンス率が0.7）
    compliance_rate = 0.3
    D = (Z * compliance_rate + np.random.normal(0.5, 0.5, size=N)).round().clip(0, 1)
    
    # アウトカム変数Y: LATEの影響をシミュレート
    true_LATE = 2  # 真のLATE値
    Y = true_LATE * D + np.random.normal(0, 1, size=N)
    
    # データをまとめる
    data = pd.DataFrame({'Z': Z, 'D': D, 'Y': Y})
    
    # 条件付き期待値を計算
    mean_Y_Z1 = data.loc[data['Z'] == 1, 'Y'].mean()
    mean_Y_Z0 = data.loc[data['Z'] == 0, 'Y'].mean()
    mean_D_Z1 = data.loc[data['Z'] == 1, 'D'].mean()
    mean_D_Z0 = data.loc[data['Z'] == 0, 'D'].mean()
    
    # LATE推定量
    estimated_LATE = (mean_Y_Z1 - mean_Y_Z0) / (mean_D_Z1 - mean_D_Z0)
    
    # 標準誤差の計算
    var_Y_Z1 = data.loc[data['Z'] == 1, 'Y'].var()
    var_Y_Z0 = data.loc[data['Z'] == 0, 'Y'].var()
    var_D_Z1 = data.loc[data['Z'] == 1, 'D'].var()
    var_D_Z0 = data.loc[data['Z'] == 0, 'D'].var()
    
    SE_LATE = np.sqrt(
        (var_Y_Z1 / len(data[data['Z'] == 1]) + var_Y_Z0 / len(data[data['Z'] == 0])) / (mean_D_Z1 - mean_D_Z0)**2 +
        (mean_Y_Z1 - mean_Y_Z0)**2 *
        (var_D_Z1 / len(data[data['Z'] == 1]) + var_D_Z0 / len(data[data['Z'] == 0])) / (mean_D_Z1 - mean_D_Z0)**4
    )
    
    # t統計量の計算
    t_stat = estimated_LATE / SE_LATE
    # p_value = 2 * (1 - stats.t.cdf(np.abs(t_stat), df=N-2))  # 両側検定
    p_value = 1 - stats.t.cdf(t_stat, df=N-2)  # 片側検定 (LATE > 0)
    
    if p_value < p_value_thres:
        significant_flag = 1
    significant_list.append(significant_flag)
significant_ratio = np.array(significant_list).mean()
print(f'有意と判定している割合: {significant_ratio:,.3f}')

100%|██████████████████████████████████████████████████████████████████████████████████████| 10000/10000 [00:11<00:00, 856.88it/s]

有意と判定している割合: 0.963


- compliance_rate = 0.2 -> 0.686
- compliance_rate = 0.3 -> 0.963